## Use Case 1
* Extract file properties from all parts, put them into an excel file
* Modify the excel file
* Extract data from modified excel file, use this to modify data in part files.
* Generate the modified BOM (done within Solidworks and not using python)

## Use Case 2
* Changing property values in multiple part files (currently supports same values only)
* Eg. 10 part files need their project names changed

In [4]:
import pySldWrap.sw_tools as sw_tools
import importlib
import os
from pathlib import Path
import time 

In [5]:

# !Solidworks must be open before running the code below: 

#reloads sw_tools module so changes in .py file actually work without restarting the kernel when running the cell blocks below
#reload causes all variables to be lost; do not do it in other cells besides this block
importlib.reload(sw_tools)
sw_tools.connect_sw("2024")  # open connection and pass Solidworks version

### Code to retrieve file properties for a **single part**

In [95]:
'''
Code for a single part
'''
# part_path = './Test_files\LS3.SLDPRT'
# model = sw_tools.open_part(part_path)  # open the model, link is returned
# custom_properties = sw_tools.get_custom_file_properties(part_path)

# a = sw_tools.export_custom_file_properties(custom_properties)

'\nCode for a single part\n'

### Code to retrieve file properties for **multiple parts**

In [50]:
importlib.reload(sw_tools)
sw_tools.connect_sw("2024")

directory = "./Test_files"
part_path = ''

# start_time = time.perf_counter()

for path, folders, files in os.walk(directory):
    for filename in files:
        # check for part files and assemblies
        try:
            if filename.endswith(('.SLDPRT','.SLDASM')):
                part_path = os.path.join(directory, filename)
                model = sw_tools.open_part(part_path)
                custom_properties = sw_tools.get_custom_file_properties(model)
                sw_tools.export_custom_file_properties(custom_properties)
                #close files to reduce memory taken up; essential for reducing time taken for whole process
                sw_tools.close(part_path.split('\\')[-1])
        except:
            continue

# end_time = time.perf_counter()
# execution_time = end_time - start_time
# print(f"Execution time: {execution_time:.2f} seconds")

### Code to modify values in solidworks part file/assembly using excel file

In [69]:
importlib.reload(sw_tools)
sw_tools.connect_sw("2024")

# user will input absolute location? of excel file
# user will input absoute location? of part files and assemblies
import pandas as pd

directory = "./Test_files"
# part_path = ''

def modify_file_properties_from_excel(directory,filename):
    '''
    Reads data from an existing BOM in an excel file
    Reads names and values of file properties in excel file
    Checks these against names and values in part/assembly files
    Overwrites them
    '''
    # *create a for loop that does this for every name in the title column:
    # *read the title and associate it with a name of a part file or assembly; if title contains assembly then its an assembly; if not its not
    # *open the specified file or assembly
    # *get the file properties in that part file/assembly
    #replace the file properties in the part file/assembly with those in the excel file

    excel_df = pd.read_excel(filename)
    #converts a pandas series into a list
    column_list = list(excel_df.loc[:,"Title"])
    #accesses the first row to retrieve names of rows
    #replaces NAN rows read from excel with --, excluding the first 4 rows

    # Columns you want to EXCLUDE from fillna
    exclude_columns = ['Enterprise Part No.', 'Title', 'V_Name', 'Revision']
    # Select columns to fill (all columns EXCEPT the excluded ones)
    columns_to_fill = [col for col in excel_df.columns if col not in exclude_columns]
    # Apply fillna() to the selected columns
    excel_df[columns_to_fill] = excel_df[columns_to_fill].fillna('--')

    new_excel_df = excel_df.set_index(excel_df.columns[0])

    for i in range(excel_df.shape[0]):
        #checking if its a part file or assembly file
        if ("Assembly" or "Assem") in column_list[i]:               # *remember to put brackets for OR conditions
            filename = column_list[i] + '.SLDASM'
        else:
            filename = column_list[i] + '.SLDPRT'

        property_value_list = list(new_excel_df.loc[column_list[i],:])

        # try:
        #open part/assembly and get custom file properties
        part_path = os.path.join(directory, filename)
        model = sw_tools.open_part(part_path)
        sw_tools.set_file_properties(model,property_value_list)
        # sw_tools.save_model(model)
        # print(sw_tools.get_custom_file_properties(model)[0].value[1],type(sw_tools.get_custom_file_properties(model)[0].value))
        sw_tools.close(part_path.split('\\')[-1])
        # except:
        #     #replace with error message to user?
        #     print('error')
        #     continue
        
        # !temp breakpoint here, remove after testing is done
        


# !need to make file properties modifiable; can be entered by user
modify_file_properties_from_excel(directory,'custom_properties.xlsx')



In [ ]:

# !replace nan values for empty cells later?

In [1]:
string1 = 'abc'
string2 = 'bd'

print(string1+string2)

abcbd
